In [4]:
from spotify_functions import *
import pandas as pd

In [2]:
artists_ids_df = pd.read_csv("artist_ids.csv")

In [1]:
# python
from spotify_functions import *
import pandas as pd

artists_ids_df = pd.read_csv("artist_ids.csv")

artists_data = []
for index, row in artists_ids_df.iterrows():
    artist_name = row['artist']
    artist_id = row['artist_id']
    if pd.isna(artist_id):
        print(f"Skipping {artist_name} due to missing artist ID.")
        continue
    print(f"Processing Artist: {artist_name}, Spotify ID: {artist_id}")
    try:
        artist_info = get_artist_info(artist_id, access_token)

        # Normalize artist_info (handle list or nested structures)
        if isinstance(artist_info, list) and artist_info:
            artist_info = artist_info[0]
        elif isinstance(artist_info, dict) and isinstance(artist_info.get('artists'), list) and artist_info['artists']:
            artist_info = artist_info['artists'][0]

        followers_count = None
        if isinstance(artist_info, dict):
            followers = artist_info.get('followers', {})
            if isinstance(followers, dict):
                followers_count = followers.get('total')

        # Get albums and normalize to a list
        albums_resp = get_artist_albums(artist_id, access_token)
        if isinstance(albums_resp, dict) and isinstance(albums_resp.get('items'), list):
            albums_list = albums_resp['items']
        elif isinstance(albums_resp, list):
            albums_list = albums_resp
        else:
            albums_list = []

        album_names = [a.get('name') for a in albums_list]
        album_release_dates = [a.get('release_date') for a in albums_list]

        tracks = []
        for album in albums_list:
            album_id = album.get('id') if isinstance(album, dict) else None
            if not album_id:
                continue

            album_tracks_resp = get_album_tracks(album_id, access_token)
            # Normalize album tracks (dict with 'items' or list)
            if isinstance(album_tracks_resp, dict) and isinstance(album_tracks_resp.get('items'), list):
                tracks_list = album_tracks_resp['items']
            elif isinstance(album_tracks_resp, list):
                tracks_list = album_tracks_resp
            else:
                tracks_list = []

            for track in tracks_list:
                if isinstance(track, dict) and 'name' in track:
                    tracks.append(track['name'])

        artists_data.append({
            'artist': artist_name,
            'artist_id': artist_id,
            'followers_count': followers_count,
            'albums': album_names,
            'album_release_dates': album_release_dates,
            'tracks': tracks
        })
        print(f"Collected data for {artist_name}: {len(album_names)} albums, {len(tracks)} tracks, {followers_count} followers.")
    except Exception as e:
        print(f"Error processing {artist_name}: {e}")

Processing Artist: Jeff Tweedy, Spotify ID: 2rDxtYUzTAYJJE3Bl3Z5IN
Error processing Jeff Tweedy: name 'pd' is not defined
Processing Artist: Geese, Spotify ID: 0WCo84qtCKfbyIf1lqQWB4
Error processing Geese: name 'pd' is not defined
Processing Artist: Piotr Kurek, Spotify ID: 0mneo6UHjcOtZBm1Tw8t67
Error processing Piotr Kurek: name 'pd' is not defined
Processing Artist: Paul St. Hilaire, Spotify ID: 2z6qOkQVyn7h3XzUuNlRlM
Error processing Paul St. Hilaire: name 'pd' is not defined
Processing Artist: Ø, Spotify ID: 0dgJbQ0bKPyUXco8hEXN7X
Error processing Ø: name 'pd' is not defined
Processing Artist: Frost Children, Spotify ID: 6R1kfr0GIWnwxY4zW11Vag
Error processing Frost Children: name 'pd' is not defined
Processing Artist: múm, Spotify ID: 4mw86zm4QZIL8SksdyE6OU
Error processing múm: name 'pd' is not defined
Processing Artist: Cardi B, Spotify ID: 4kYSro6naA4h99UJvo89HB
Error processing Cardi B: name 'pd' is not defined
Processing Artist: Asher White, Spotify ID: 1TQhvHMVoECTNs3Xxo3R

KeyboardInterrupt: 

In [3]:
# Loop through each artist ID and get their albums, follower count, songs and save to a new CSV file
artists_data = [] # List to hold data for all artists
for index, row in artists_ids_df.iterrows(): # Iterate over each artist
    artist_name = row['artist'] # Get artist name from the DataFrame
    artist_id = row['artist_id']
    if pd.isna(artist_id):
        print(f"Skipping {artist_name} due to missing artist ID.") # Skip if artist ID is missing (NaN)
        continue
    print(f"Processing Artist: {artist_name}, Spotify ID: {artist_id}")
    try:
        artist_info = get_artist_info(artist_id, access_token)
        followers_count = artist_info['followers']['total']
        albums = get_artist_albums(artist_id, access_token)
        album_names = [album['name'] for album in albums]
        album_release_dates = [album['release_date'] for album in albums]
        tracks = []
        for album in albums:
            album_id = album['id']
            album_tracks = get_album_tracks(album_id, access_token)
            for track in album_tracks:
                tracks.append(track['name'])
        artists_data.append({
            'artist': artist_name,
            'artist_id': artist_id,
            'followers_count': followers_count,
            'albums': album_names,
            'album_release_dates': album_release_dates,
            'tracks': tracks
        })
        print(f"Collected data for {artist_name}: {len(albums)} albums, {len(tracks)} tracks, {followers_count} followers.")
    except Exception as e:
        print(f"Error processing {artist_name}: {e}")

Processing Artist: Jeff Tweedy, Spotify ID: 2rDxtYUzTAYJJE3Bl3Z5IN
Error processing Jeff Tweedy: name 'pd' is not defined
Processing Artist: Geese, Spotify ID: 0WCo84qtCKfbyIf1lqQWB4
Error processing Geese: name 'pd' is not defined
Processing Artist: Piotr Kurek, Spotify ID: 0mneo6UHjcOtZBm1Tw8t67
Error processing Piotr Kurek: name 'pd' is not defined
Processing Artist: Paul St. Hilaire, Spotify ID: 2z6qOkQVyn7h3XzUuNlRlM
Error processing Paul St. Hilaire: name 'pd' is not defined
Processing Artist: Ø, Spotify ID: 0dgJbQ0bKPyUXco8hEXN7X
Error processing Ø: name 'pd' is not defined
Processing Artist: Frost Children, Spotify ID: 6R1kfr0GIWnwxY4zW11Vag
Error processing Frost Children: name 'pd' is not defined
Processing Artist: múm, Spotify ID: 4mw86zm4QZIL8SksdyE6OU
Error processing múm: name 'pd' is not defined
Processing Artist: Cardi B, Spotify ID: 4kYSro6naA4h99UJvo89HB
Error processing Cardi B: name 'pd' is not defined
Processing Artist: Asher White, Spotify ID: 1TQhvHMVoECTNs3Xxo3R

KeyboardInterrupt: 